In [ ]:
from pathlib import Path

import cairosvg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skunk
import yaml
from coulson.data import EV_TO_KCAL, HARTREE_TO_EV, HARTREE_TO_KCAL
from coulson.draw import draw_mol, draw_orbital_energies
from coulson.interface import gen_coords_for_mol, mol_from_xyz, process_rdkit_mol
from coulson.ppp import (
    PPPCalculator,
    calculate_dsp,
    calculate_exchange,
    homo_lumo_overlap,
)
from rdkit import Chem
from rdkit.Chem import AllChem
from utils import crop_image

plt.style.use("default")

In [ ]:
mol_azaphenalene = mol_from_xyz(snakemake.input.azaphenalene)
mol_pentalene = mol_from_xyz(snakemake.input.pentalene)
mol_pentalene_ideal = mol_from_xyz(snakemake.input.pentalene)
_ = gen_coords_for_mol(mol_pentalene_ideal)

In [ ]:
def generate_orbital_figure(mol_planar, ppp, exchange):
    # Generate orbital figures
    fig_orbitals, ax_orbitals = draw_orbital_energies(
        energies=ppp.orbital_energies * HARTREE_TO_EV,
        occupations=ppp.occupations,
        axis_label="Orbital energy (eV)",
        invert_axis=False,
        occupation_labels=range(1, ppp.n_orbitals + 1),
        draw_occupation_labels=True,
    )
    # Create image of the frontier orbitals
    svg_homo = draw_mol(
        mol_planar, properties=ppp.coefficients[ppp.homo_idx], img_format="svg"
    )
    svg_lumo = draw_mol(
        mol_planar, properties=ppp.coefficients[ppp.lumo_idx], img_format="svg"
    )

    # Create combined image with orbital energies and frontier orbitals
    svg_lobes = skunk.layout_svgs([svg_homo, svg_lumo], labels=["HOMO", "LUMO"])
    plt.close()

    label_exchange = (
        "Frontier orbitals\n"
        + "$_{"
        + f"2K={2 * exchange * HARTREE_TO_KCAL:.2f}"
        + "\/\mathrm{kcal/mol}}$"
    )

    shape = (1, 2)
    figsize = (shape[1] * 2 * 1.5, shape[0] * 3 * 1.5)
    svg_orbitals = skunk.layout_svgs(
        [fig_orbitals, svg_lobes],
        labels=["Orbital energies\n", label_exchange],
        shape=shape,
        figsize=figsize,
    )
    plt.close()
    png = cairosvg.svg2png(bytestring=svg_orbitals, dpi=600, background_color="white")
    png_cropped = crop_image(png)

    # with open("orbitals.png", "wb") as f:
    #     f.write(png_cropped)
    return png_cropped


def generate_excitation_figure(mol_planar, ppp, excitations):
    svgs = []
    labels = []
    for (i, j), data in excitations.items():
        coefficients = np.zeros(mol.GetNumAtoms())
        img_occupied = draw_mol(
            mol_planar, properties=ppp.coefficients[i], img_format="svg"
        )
        img_virtual = draw_mol(
            mol_planar, properties=ppp.coefficients[j], img_format="svg"
        )
        svg = skunk.layout_svgs(
            [img_occupied, img_virtual], labels=["Occupied", "Virtual"]
        )
        plt.close()

        svgs.append(svg)
        label = (
            f"{i + 1} → {j + 1}\n"
            + "$_{"
            + f"{data['dsp'] * HARTREE_TO_KCAL:.2f}"
            + "\/\mathrm{kcal/mol}}$"
        )

        labels.append(label)

    svg = skunk.layout_svgs(svgs, labels=labels)
    plt.close()
    png = cairosvg.svg2png(bytestring=svg, dpi=300, background_color="white")
    png_cropped = crop_image(png)
    return png_cropped

In [ ]:
def compute_mol(mol):
    input_data, _ = process_rdkit_mol(mol)

    # Do SCF calculation
    ppp = PPPCalculator(input_data)
    ppp.scf()

    # Calcualte exchange integral
    exchange = calculate_exchange(ppp)

    # Calculate DSP contribution
    dsp_scf, excitations_scf = calculate_dsp(ppp)

    # Calculate HOMO-LUMO overlap
    overlap = homo_lumo_overlap(ppp)

    # Calculate S1 energies and oscillator strengths
    ppp.ci(n_states=3)
    energy_s_1_cis = ppp.ci_energies[0] - ppp.electronic_energy

    oscillator_strength = ppp.oscillator_strengths[0]

    # Calculate T1 energies
    ppp.ci(n_states=3, multiplicity="triplet")
    energy_t_1_cis = ppp.ci_energies[0] - ppp.electronic_energy

    dsp_cis, _ = calculate_dsp(
        ppp, ci=True, energy_s_1=energy_s_1_cis, energy_t_1=energy_t_1_cis
    )

    t1_s1_cis = energy_s_1_cis - energy_t_1_cis
    t1_s1_dsp_cis = t1_s1_cis + dsp_cis
    t1_s1_dsp_scf = 2 * exchange + dsp_scf

    results = {
        "t1_s1_cis": f"{t1_s1_cis * HARTREE_TO_KCAL:.2f}",
        "t1_s1_dsp_cis": f"{t1_s1_dsp_cis * HARTREE_TO_KCAL:.2f}",
        "t1_s1_dsp_scf": f"{t1_s1_dsp_scf * HARTREE_TO_KCAL:.2f}",
        "2_exchange": f"{2 * exchange * HARTREE_TO_KCAL:.2f}",
        "dsp_cis": f"{dsp_cis * HARTREE_TO_KCAL:.2f}",
        "dsp_scf": f"{dsp_scf * HARTREE_TO_KCAL:.2f}",
        "overlap": f"{overlap:.2f}",
        "oscillator_strength": f"{oscillator_strength:.3f}",
    }

    # Create planar mol for plotting
    mol_planar = Chem.Mol(mol)
    AllChem.Compute2DCoords(mol_planar)
    mol_planar = AllChem.RemoveHs(mol_planar)

    # Generate PNG images
    png_orbitals = generate_orbital_figure(mol_planar, ppp, exchange)
    png_excitations = generate_excitation_figure(mol_planar, ppp, excitations_scf)

    return results, png_orbitals, png_excitations

In [ ]:
jobs = (
    (
        "azaphenalene",
        mol_azaphenalene,
        snakemake.output.azaphenalene_orbitals,
        snakemake.output.azaphenalene_excitations,
    ),
    (
        "pentalene",
        mol_pentalene,
        snakemake.output.pentalene_orbitals,
        snakemake.output.pentalene_excitations,
    ),
    (
        "pentalene_ideal",
        mol_pentalene_ideal,
        snakemake.output.pentalene_ideal_orbitals,
        snakemake.output.pentalene_ideal_excitations,
    ),
)
variables = {}
for name, mol, path_orbitals, path_excitations in jobs:
    results, png_orbitals, png_excitations = compute_mol(mol)

    # Write png files to disk
    # fig_path_orbitals = f"../../results1/figures/{name}_orbitals.png"
    with open(path_orbitals, "wb") as f:
        f.write(png_orbitals)
    # fig_path_excitations = f"../../results1/figures/{name}_excitations.png"
    with open(path_excitations, "wb") as f:
        f.write(png_excitations)

    # Collect all variables
    for key, value in results.items():
        variables[f"{name}_{key}"] = value
    variables[f"fig_{name}_orbitals"] = "../" + str(Path(path_orbitals).with_suffix(""))
    variables[f"fig_{name}_excitations"] = "../" + str(
        Path(path_excitations).with_suffix("")
    )

Take out reference data

In [ ]:
# snakemake.input.ref_azaphenalene
df = pd.read_csv(snakemake.input.azaphenalene_ref, index_col=0)
variables["azaphenalene_t1_s1_ref"] = f"{df.loc[1]['t1_s1_ref'] * EV_TO_KCAL:.2f}"

# snakemake.input.ref_pentalene
df = pd.read_csv(snakemake.input.pentalene_ref, index_col=0)
variables["pentalene_t1_s1_ref"] = f"{df.loc['IV_1']['t1_s1_ref'] * EV_TO_KCAL:.2f}"

Write everything YAML for processing by article

In [ ]:
with open(snakemake.output.variables, "w") as f:
    yaml.dump(variables, f)